# Draft Assistant

A code-based draft assistant (no UI).

In [1]:
# Install package
%pip install .. -q

import pandas as pd
import statisticaldrafting as sd

Note: you may need to restart the kernel to use updated packages.


In [2]:
# Load a model. 
# sd.list_sets() # Show available sets.
dm = sd.DraftModel(set="FDN", draft_mode="Premier")

In [4]:
# collection = ["Vivien Reid", "Scavenging Ooze"] # Collection as cardnames
# collection = [16, 229, 78, 78, 198] # Collection as card ids
collection = [54]
pick_order = dm.get_pick_order(collection)
# pick_order = pick_order[pick_order["rarity"].isin(["common", "uncommon"])] # Optional filter. 
# pick_order = pick_order[pick_order["color_identity"].isin(["G"])] # Optional filter. 
pick_order

,name,rarity,color_identity,synergy,p1p1_rating,rating
223,Sire of Seven Deaths,mythic,Colorless,0.00,5.00,5.00
54,"Chandra, Flameshaper",mythic,R,0.16,4.46,4.62
85,Embercleave,special,R,0.03,4.38,4.41
162,"Liliana, Dreadhorde General",mythic,B,0.04,4.35,4.39
35,Bloodthirsty Conqueror,mythic,B,0.08,4.29,4.37
263,Twinflame Tyrant,mythic,R,0.26,3.99,4.25
7,"Ajani, Caller of the Pride",mythic,W,-0.15,4.32,4.17
271,Vivien Reid,mythic,G,0.22,3.88,4.10
155,"Kiora, the Rising Tide",rare,U,-0.03,4.13,4.10
236,Spinner of Souls,rare,G,0.30,3.77,4.07


In [ ]:
# Deckbuild recommendations after draft. 
# dm.get_deck_recommendation(collection, starting_colors="")